# Classification Dataset
BS"D

In this notebook, I will put together the dataset of food items labeled according to dietary restrictions. It will be used to train classifiers for each dietary restriction.

The diets I will be working on are:
- Vegetarian
- Gluten Free
- Dairy Free

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Hand-Labeling Data
The following set of ingredients will be hand-labeled according to the dietary restrictions.

In [29]:
def load_common_ingredients(file_name = "source_data/common_ingredients.json"):
    # Load the data
    data = pd.read_json(file_name)

    # The data is a dataframe of dictionaries and needs to be exploded
    data["ingredient"] = data["data"].apply(lambda x: x["index"])
    data["quantity"] = data["data"].apply(lambda x: x["quantity"])

    # Drop the data column
    data = data.drop("data", axis=1)

    return data

In [31]:
common_ingredients = load_common_ingredients()

common_ingredients

,ingredient,quantity
0,salt,18049
1,olive oil,7972
2,onions,7972
3,water,7457
4,garlic,7380
...,...,...
495,boneless chicken breast,146
496,crème fraîche,145
497,cooked white rice,145
498,pecans,144


In [34]:
# Save the data as an excel file
with pd.ExcelWriter("common_ingredients.xlsx") as writer:
    common_ingredients.to_excel(writer, sheet_name="common_ingredients", index=False)

